# Tutorial: Die grundlegenden Werkzeuge des privaten Deep Learning

Willkommen zu PySyfts Einführungs-Tutorial zum Schutz der Privatsphäre und zum dezentralen Deep Learning. Diese Reihe von Notebook's ist eine schrittweise Anleitung, mit der Sie die neuen Tools und Techniken kennenlernen können, die für das gründliche Lernen geheimer / privater Daten / Modelle erforderlich sind, ohne sie unter einer Autorität zu zentralisieren.

** Geltungsbereich: ** Beachten Sie, dass wir nicht nur über das Dezentralisieren / Verschlüsseln von Daten sprechen, sondern auch darüber, wie PySyft verwendet werden kann, um das gesamte Ökosystem um Daten herum zu dezentralisieren, einschließlich der Datenbanken, in denen sich Daten befinden gespeichert und abgefragt sowie die neuronalen Modelle, mit denen Informationen aus Daten extrahiert werden. Wenn neue Erweiterungen für PySyft erstellt werden, werden diese notebook's um neue Tutorials erweitert, um die neuen Funktionen zu erläutern.

Autoren:
- Andrew Trask - Twitter: [@iamtrask](https://twitter.com/iamtrask)

Übersetzer:
- Vineet Jain - Github: [@vineetjai](https://github.com/vineetjai)

## Gliederung:

- Teil 1: Die grundlegenden Werkzeuge des privaten Deep Learning


## Warum dieses Tutorial machen?

** 1) Ein wettbewerbsfähiger Karrierevorteil ** - In den letzten 20 Jahren hat die digitale Revolution die Daten in immer größeren Mengen zugänglicher gemacht, da analoge Prozesse digitalisiert wurden. Mit neuen Regelungen wie z [GDPR](https://eugdpr.org/),Unternehmen stehen unter dem Druck, weniger Freiheit bei der Verwendung - und vor allem bei der Analyse - persönlicher Informationen zu haben. ** Fazit: ** Data Scientists werden mit "Old School" -Tools nicht auf so viele Daten zugreifen können, aber durch das Erlernen der Tools von Private Deep Learning können SIE dieser Kurve voraus sein und einen Wettbewerbsvorteil erzielen Deine Karriere.

** 2) Unternehmerische Möglichkeiten ** - Es gibt eine ganze Reihe von Problemen in der Gesellschaft, die Deep Learning lösen kann, aber viele der wichtigsten wurden nicht untersucht, da dies den Zugriff auf unglaublich sensible Informationen über Menschen erfordern würde (erwägen Sie die Verwendung von Deep Lernen, Menschen mit mentalen oder Beziehungsproblemen zu helfen!). Das Erlernen von Private Deep Learning eröffnet Ihnen somit eine Vielzahl neuer Startmöglichkeiten, die anderen ohne diese Toolsets bisher nicht zur Verfügung standen.

** 3) Soziales Gut ** - Deep Learning kann verwendet werden, um eine Vielzahl von Problemen in der realen Welt zu lösen, aber Deep Learning über * persönliche Informationen * ist Deep Learning über Menschen, * für Menschen *. Das Erlernen des Deep Learning für Daten, die Sie nicht besitzen, ist mehr als eine berufliche oder unternehmerische Chance. Es ist die Gelegenheit, einige der persönlichsten und wichtigsten Probleme im Leben der Menschen zu lösen - und dies in großem Maßstab.

## Wie bekomme ich zusätzliches Guthaben?

- Star PySyft auf GitHub! - [https://github.com/OpenMined/PySyft](https://github.com/OpenMined/PySyft)
- Mach ein Youtube-Video, in dem dieses Notebook's unterrichtet wird!


... Ok, lasst uns das machen!

# Teil -1: Voraussetzungen

- Kennen Sie PyTorch - wenn nicht, nehmen Sie am http://fast.ai Kurs teil und kommen Sie zurück
- Lesen Sie das PySyft Framework Paper https://arxiv.org/pdf/1811.04017.pdf! Auf diese Weise erhalten Sie einen umfassenden Hintergrund zum Aufbau von PySyft, wodurch die Dinge sinnvoller werden.


# Teil 0: Setup

Zu Beginn müssen Sie sicherstellen, dass Sie die richtigen Dinge installiert haben. Gehen Sie dazu zu PySyfts Readme und folgen Sie den Anweisungen zum Einrichten. TLDR für die meisten Leute ist.

- Installieren Sie Python 3.6 oder höher
- Installieren Sie PyTorch 1.3
- Clone PySyft (git clone https://github.com/OpenMined/PySyft.git)
- cd PySyft
- pip install -r pip-dep/requirements.txt
- pip install -r pip-dep/requirements_udacity.txt
- python setup.py install udacity
- python setup.py test


Wenn ein Teil davon für Sie nicht funktioniert (oder einer der Tests fehlschlägt), überprüfen Sie zuerst die [README](https://github.com/OpenMined/PySyft.git) auf Installationshilfe und öffnen Sie dann ein GitHub-Problem oder pingen Sie den #beginner kanal in unserem Slack! [slack.openmined.org](http://slack.openmined.org/)

In [1]:
# Run this cell to see if things work
import sys

import torch
from torch.nn import Parameter
import torch.nn as nn
import torch.nn.functional as F

import syft as sy
hook = sy.TorchHook(torch)

torch.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

Wenn diese Zelle ausgeführt wird, geht es los zu den Rennen! Lass uns das machen!

# Teil 1: Die grundlegenden Werkzeuge der privaten, dezentralen Datenwissenschaft

Die erste Frage, die Sie sich vielleicht fragen, lautet: Wie in aller Welt trainieren wir ein Modell für Daten, auf die wir keinen Zugriff haben?

Die Antwort ist überraschend einfach. Wenn Sie es gewohnt sind, in PyTorch zu arbeiten, sind Sie es gewohnt, mit torch zu arbeiten. Tensor-Objekte wie diese!

In [2]:
x = torch.tensor([1,2,3,4,5])
y = x + x
print(y)

tensor([ 2,  4,  6,  8, 10])


Natürlich ist die Verwendung dieser ausgefallenen (und leistungsstarken!) Tensoren wichtig, erfordert aber auch, dass Sie die Daten auf Ihrem lokalen Computer haben. Hier beginnt unsere Reise.

# Abschnitt 1.1 - Senden von Tensoren an Bobs Maschine

Während wir normalerweise Data Science / Deep Learning auf der Maschine durchführen würden, die die Daten enthält, möchten wir diese Art der Berechnung jetzt auf einer ** anderen ** Maschine durchführen. Insbesondere können wir nicht mehr einfach davon ausgehen, dass sich die Daten auf unserem lokalen Computer befinden.

Anstatt PyTorch tensors zu verwenden, werden wir jetzt mit ** Zeigern ** auf Tensoren arbeiten. Lass mich dir zeigen, was ich meine. Lassen Sie uns zunächst eine "vorgetäuschte" Maschine erstellen, die einer "vorgetäuschten" Person gehört - wir nennen sie Bob.

In [3]:
bob = sy.VirtualWorker(hook, id="bob")

Nehmen wir an, Bobs Maschine befindet sich auf einem anderen Planeten - vielleicht auf dem Mars! Aber im Moment ist die Maschine leer. Lassen Sie uns einige Daten erstellen, damit wir sie an Bob senden und mehr über Zeiger erfahren können!

In [4]:
x = torch.tensor([1,2,3,4,5])
y = torch.tensor([1,1,1,1,1])

Und jetzt - lass uns unsere Tensoren an Bob schicken !!

In [6]:
x_ptr = x.send(bob)
y_ptr = y.send(bob)

In [7]:
x_ptr

(Wrapper)>[PointerTensor | me:83083126699 -> bob:57691089902]

BOOM! Jetzt hat Bob zwei Tensoren! Glaubst du mir nicht? Überzeugen Sie sich selbst!

In [8]:
bob._objects

{57691089902: tensor([1, 2, 3, 4, 5]), 78260268274: tensor([1, 1, 1, 1, 1])}

In [9]:
z = x_ptr + x_ptr

In [10]:
z

(Wrapper)>[PointerTensor | me:39740619599 -> bob:55000474416]

In [11]:
bob._objects

{57691089902: tensor([1, 2, 3, 4, 5]),
 78260268274: tensor([1, 1, 1, 1, 1]),
 55000474416: tensor([ 2,  4,  6,  8, 10])}

Jetzt merke etwas. Als wir "x.send (bob)" aufriefen, gab es ein neues Objekt zurück, das wir "x_ptr" nannten. Dies ist unser erster * Zeiger * auf einen Tensor. Zeiger auf Tensoren enthalten KEINE Daten. Stattdessen enthalten sie einfach Metadaten zu einem Tensor (mit Daten), die auf einem anderen Computer gespeichert sind. Der Zweck dieser Tensoren besteht darin, uns eine intuitive API zu geben, mit der die andere Maschine angewiesen wird, Funktionen mit diesem Tensor zu berechnen. Werfen wir einen Blick auf die Metadaten, die Zeiger enthalten.

In [12]:
x_ptr

(Wrapper)>[PointerTensor | me:83083126699 -> bob:57691089902]

Schauen Sie sich diese Metadaten an!

Es gibt zwei Hauptattribute, die für Zeiger spezifisch sind:

- `x_ptr.location : bob`, der Ort, ein Verweis auf den Ort, auf den der Zeiger zeigt
- `x_ptr.id_at_location : <random integer>`, die ID, in der der Tensor am Ort gespeichert ist

Sie werden im Format "<id_at_location>@<location>" gedruckt

Es gibt auch andere allgemeinere Attribute:
- `x_ptr.id : <random integer>`, die ID unseres Zeigertensors, wurde zufällig zugewiesen
- `x_ptr.owner : "me"`, der Arbeiter, dem der Zeigertensor gehört, hier ist es der lokale Arbeiter mit dem Namen "ich"

In [13]:
x_ptr.location

<VirtualWorker id:bob #objects:3>

In [14]:
bob

<VirtualWorker id:bob #objects:3>

In [15]:
bob==x_ptr.location

True

In [16]:
x_ptr.id_at_location

57691089902

In [17]:
x_ptr.owner

<VirtualWorker id:me #objects:0>

Sie fragen sich vielleicht, warum der lokale Mitarbeiter, dem der Zeiger gehört, auch ein VirtualWorker ist, obwohl wir ihn nicht erstellt haben.
Unterhaltsame Tatsache, genau wie wir ein VirtualWorker-Objekt für Bob hatten, haben wir (standardmäßig) immer auch eines für uns. Dieser Worker wird automatisch erstellt, wenn wir "hook = sy.TorchHook ()" aufrufen. Daher müssen Sie ihn normalerweise nicht selbst erstellen.

In [18]:
me = sy.local_worker
me

<VirtualWorker id:me #objects:0>

In [19]:
me == x_ptr.owner

True

Und schließlich können wir, genau wie wir .send() für einen Tensor aufrufen können, .get() für einen Zeiger auf einen Tensor aufrufen, um ihn zurückzubekommen !!!

In [20]:
x_ptr

(Wrapper)>[PointerTensor | me:83083126699 -> bob:57691089902]

In [21]:
x_ptr.get()

tensor([1, 2, 3, 4, 5])

In [22]:
y_ptr

(Wrapper)>[PointerTensor | me:17829909481 -> bob:78260268274]

In [23]:
y_ptr.get()

tensor([1, 1, 1, 1, 1])

In [24]:
z.get()

tensor([ 2,  4,  6,  8, 10])

In [25]:
bob._objects

{}

Und wie Sie sehen können ... Bob hat die Tensoren nicht mehr !!! Sie sind zurück zu unserer Maschine gezogen!

# Abschnitt 1.2 - Verwenden von Tensorzeigern

Das Senden und Empfangen von Tensoren von Bob ist also großartig, aber das ist kaum Deep Learning! Wir möchten in der Lage sein, Tensoroperationen an entfernten Tensoren durchzuführen. Glücklicherweise machen Tensorzeiger dies ziemlich einfach! Sie können einfach Zeiger wie normale Tensoren verwenden!

In [26]:
x = torch.tensor([1,2,3,4,5]).send(bob)
y = torch.tensor([1,1,1,1,1]).send(bob)

In [27]:
z=x+y

In [28]:
z

(Wrapper)>[PointerTensor | me:12028202952 -> bob:76071755840]

Und voilà!

Hinter den Kulissen passierte etwas sehr Mächtiges. Anstatt x und y eine Addition lokal zu berechnen, wurde ein Befehl serialisiert und an Bob gesendet, der die Berechnung durchführte, einen Tensor z erstellte und dann den Zeiger auf z an uns zurückgab!

Wenn wir .get() für den Zeiger aufrufen, erhalten wir das Ergebnis zurück an unsere Maschine!

In [29]:
z.get()

tensor([2, 3, 4, 5, 6])

### Brennerfunktionen

Diese API wurde auf alle Operationen von Torch erweitert !!!

In [31]:
x

(Wrapper)>[PointerTensor | me:97521977565 -> bob:75592019932]

In [33]:
y

(Wrapper)>[PointerTensor | me:92075559708 -> bob:7890228100]

In [34]:
z = torch.add(x,y)
z

(Wrapper)>[PointerTensor | me:71694644898 -> bob:48970175826]

In [35]:
z.get()

tensor([2, 3, 4, 5, 6])

### Variablen (einschließlich Backpropagation!)

In [36]:
x = torch.tensor([1,2,3,4,5.], requires_grad=True).send(bob)
y = torch.tensor([1,1,1,1,1.], requires_grad=True).send(bob)

In [37]:
z = (x + y).sum()

In [38]:
z.backward()

(Wrapper)>[PointerTensor | me:22226407463 -> bob:57302226904]

In [39]:
x = x.get()

In [40]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [41]:
x.grad

tensor([1., 1., 1., 1., 1.])

Wie Sie sehen können, ist die API sehr flexibel und kann nahezu alle Vorgänge ausführen, die Sie normalerweise in Torch für * Remote-Daten * ausführen würden. Dies bildet die Grundlage für unsere fortschrittlicheren Protokolle zur Wahrung der Privatsphäre wie Federated Learning, Secure Multi-Party Computation und Differential Privacy!

# Herzliche Glückwünsche!!! - Zeit, der Community beizutreten!

Herzlichen Glückwunsch zum Abschluss dieses Notebook-Tutorials! Wenn Ihnen dies gefallen hat und Sie sich der Bewegung zur Wahrung der Privatsphäre, zum dezentralen Besitz von KI und der KI-Lieferkette (Daten) anschließen möchten, können Sie dies auf folgende Weise tun!

### Star PySyft auf GitHub

Der einfachste Weg, unserer Community zu helfen, besteht darin, die GitHub-Repos in der Hauptrolle zu spielen! Dies hilft, das Bewusstsein für die coolen Tools zu schärfen, die wir bauen.

- [Star PySyft](https://github.com/OpenMined/PySyft)

### Mach mit bei unserem Slack!

Der beste Weg, um über die neuesten Entwicklungen auf dem Laufenden zu bleiben, ist, sich unserer Community anzuschließen! Sie können dies tun, indem Sie das Formular unter [http://slack.openmined.org](http://slack.openmined.org) ausfüllen.

### Treten Sie einem Code-Projekt bei!

Der beste Weg, um zu unserer Community beizutragen, besteht darin, Code-Mitwirkender zu werden! Sie können jederzeit zur Seite PySyft GitHub Issues gehen und nach "Projekten" filtern. Dies zeigt Ihnen alle Top-Level-Tickets und gibt einen Überblick darüber, an welchen Projekten Sie teilnehmen können! Wenn Sie nicht an einem Projekt teilnehmen möchten, aber ein wenig programmieren möchten, können Sie auch nach weiteren "einmaligen" Miniprojekten suchen, indem Sie nach GitHub-Problemen suchen, die als "gute erste Ausgabe" gekennzeichnet sind.

- [PySyft Projects](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3AProject)
- [Good First Issue Tickets](https://github.com/OpenMined/PySyft/issues?q=is%3Aopen+is%3Aissue+label%3A%22good+first+issue%22)

### Spenden

Wenn Sie keine Zeit haben, zu unserer Codebasis beizutragen, aber dennoch Unterstützung leisten möchten, können Sie auch Unterstützer unseres Open Collective werden. Alle Spenden fließen in unser Webhosting und andere Community-Ausgaben wie Hackathons und Meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)